# Applied Data Science Capstone

*determine neighborhoods on the other side of the city that are exactly the same as your current neighborhood*

In [158]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 
import json
import requests
from pandas.io.json import json_normalize

## Segmenting and Clustering Neighborhoods in Toronto

Download html Wikipedia page

In [10]:
from urllib.request import Request, urlopen
from urllib.error import URLError

data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

req = Request(data_url)

try:
    with urlopen(req) as response:
        page_bytes = response.read()
        page = page_bytes.decode("utf8")
except URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)    
        

Declaring custom html parser. 

In [16]:
from html.parser import HTMLParser

class TableParser(HTMLParser):
    
    def __init__(self):
        
        HTMLParser.__init__(self)
        self._header = False
        self._cell = False
        self.tables = []
        self._current_cell = []
        self._current_row = []
        self._data_separator=' '
        self._current_table = []
        
    def handle_starttag(self, tag, attrs):
        if(tag == 'td'):
            self._cell = True
        if(tag == 'th'):
            self._header = True
            
    def handle_data(self, data):
        if(self._header or self._cell):
            self._current_cell.append(data.strip())
            
    def handle_charref(self, name):
        
        if self._parse_html_entities:
            self.handle_data(self.unescape('&#{};'.format(name)))
            
    def handle_endtag(self, tag):
        
        if tag == 'td':
            self._in_td = False
        elif tag == 'th':
            self._in_th = False

        if tag in ['td', 'th']:
            final_cell = self._data_separator.join(self._current_cell).strip()
            self._current_row.append(final_cell)
            self._current_cell = []
        elif tag == 'tr':
            self._current_table.append(self._current_row)
            self._current_row = []
        elif tag == 'table':
            self.tables.append(self._current_table)
            self._current_table = []

Parsing raw html text to padas dataframe

In [64]:
parser = TableParser()
parser.feed(page)

if len(parser.tables) > 0:
    table = np.array(parser.tables[0])
    df_raw = pd.DataFrame(data=table[1:,0:], columns=table[0,0:])
else:   
    print('Tables not found')

In [47]:
df_raw.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Cleaning dataset:
   * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
   * More than one neighborhood can exist in one postal code area.
   * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [77]:
df_raw = df_raw[df_raw.Borough != 'Not assigned']
df_raw.loc[df_raw.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_raw.loc[df_raw.Neighbourhood == 'Not assigned', 'Borough']
df_raw = df_raw.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

In [78]:
df_raw.shape

(103, 3)

## Latitude and longitude coordinates of each neighborhood.

Installing geocoder

In [90]:
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 19.7MB/s ta 0:00:01


In [127]:
import geocoder
import json

def  get_coordinates(postcode):
    g = geocoder.arcgis('{}, Toronto'.format(postcode))
    latitude =  g.json['lat']
    longitude = g.json['lng']
    
    return [latitude, longitude]

In [132]:
df_raw['Latitude'] = 0.0
df_raw['Longitude'] = 0.0

for index, row in df_raw.iterrows():
    coord =  get_coordinates(row['Postcode'])
    df_raw.at[index,'Latitude'] = coord[0]
    df_raw.at[index,'Longitude'] = coord[1]

df_raw.head()        

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Create a map of Toronto with neighborhoods superimposed on top.

In [135]:
address = 'Toronto, Ontario'

g = geocoder.arcgis(address)
latitude =  g.json['lat']
longitude = g.json['lng']

print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.648690000000045, -79.38543999999996.


In [213]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_raw['Latitude'], df_raw['Longitude'], df_raw['Borough'], df_raw['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto 

 ##  Neighborhoods clustering

In [144]:
CLIENT_ID = 'ZL2BU5PAKUZSZZXZ115DCBVGP5Z1ESBTSL2RI3E0JL0DB4CD'
CLIENT_SECRET = 'E00WJ4I3OFAWOFJNMMGZ1JD51N2EMW4MWZ3BKRU1IQLAWNUP' 
VERSION = '20180605' 

In [146]:
df_raw.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [148]:
neighborhood_latitude = df_raw.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_raw.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_raw.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.811525000000074, -79.19551721399995.


In [149]:
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=ZL2BU5PAKUZSZZXZ115DCBVGP5Z1ESBTSL2RI3E0JL0DB4CD&client_secret=E00WJ4I3OFAWOFJNMMGZ1JD51N2EMW4MWZ3BKRU1IQLAWNUP&v=20180605&ll=43.811525000000074,-79.19551721399995&radius=500&limit=100'

In [ ]:
results = requests.get(url).json()

In [156]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [159]:
venues = results['response']['groups'][0]['items'] 
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,R & K Woodworking Specialists Inc,Construction & Landscaping,43.808233,-79.196857
1,Canadian Appliance Source Whitby,Home Service,43.808353,-79.191331


In [160]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [161]:
toronto_venues = getNearbyVenues(names=df_raw['Neighbourhood'],
                                   latitudes=df_raw['Latitude'],
                                   longitudes=df_raw['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [162]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,R & K Woodworking Specialists Inc,43.808233,-79.196857,Construction & Landscaping
1,"Rouge, Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
2,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.765690,-79.175256,The Strawberry Patch,43.764738,-79.173081,Tea Room
4,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


In [202]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,15,15,15,15,15,15
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13
"Alderwood, Long Branch",5,5,5,5,5,5


In [164]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 261 uniques categories.


In [167]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] =toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.tail()

,Yoga Studio,ATM,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
2472,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,ATM,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.03,0.0,0.01,0.0,0.0,...,0.0,0.0,0.0,0.01,0.0,0.000000,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.066667,0.00,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0


In [169]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2        Hotel  0.04
3          Gym  0.03
4   Steakhouse  0.03


----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.13
1         Shopping Mall  0.13
2  Hong Kong Restaurant  0.07
3                Bakery  0.07
4        Discount Store  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0            Pharmacy  0.67
1    Sushi Restaurant  0.33
2         Yoga Studio  0.00
3  Miscellaneous Shop  0.00
4       Moving Target  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.15
1           Pizza Place  0.08
2        Hardware Store  0.08
3   Fried Chicken Joint  0.08
4  Fast Food Restaurant  0.08


----Alderwood, Long Branch----
                venue  freq
0   Convenience Store   0.2
1                

In [170]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [205]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Agincourt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Agincourt North, L'Amoreaux East, Milliken, St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Alderwood, Long Branch",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clustering

In [201]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering =toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 7, 0, 0, 9, 6, 0, 0, 7], dtype=int32)

In [206]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
df_raw.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)
toronto_merged = df_raw
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,1.0,Home Service,Construction & Landscaping,ATM,Afghan Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,3.0,Bar,Women's Store,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,6.0,Construction & Landscaping,Gym / Fitness Center,Tea Room,Park,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0.0,Business Service,Indian Restaurant,Korean Restaurant,Park,Coffee Shop,Event Space,Falafel Restaurant,Farm,Farmers Market,Women's Store
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,4.0,Playground,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm


In [207]:
toronto_merged.fillna(value={'Cluster Labels':0}, inplace=True)

Finally, let's visualize the resulting clusters

In [208]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [209]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0.0,Business Service,Indian Restaurant,Korean Restaurant,Park,Coffee Shop,Event Space,Falafel Restaurant,Farm,Farmers Market,Women's Store
5,Scarborough,0.0,Restaurant,Grocery Store,Indian Restaurant,Train Station,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm
6,Scarborough,0.0,Discount Store,Hobby Shop,Coffee Shop,Department Store,Women's Store,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
7,Scarborough,0.0,Bakery,Intersection,Bus Line,Coffee Shop,Bus Station,Metro Station,Soccer Field,Dessert Shop,Dim Sum Restaurant,Flower Shop
8,Scarborough,0.0,Fast Food Restaurant,Liquor Store,Sandwich Place,Discount Store,Coffee Shop,Pizza Place,Hardware Store,Burger Joint,Pharmacy,Fish & Chips Shop
10,Scarborough,0.0,Wine Shop,Gift Shop,Light Rail Station,Bakery,Women's Store,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
12,Scarborough,0.0,Shopping Mall,Chinese Restaurant,Grocery Store,Discount Store,Skating Rink,Vietnamese Restaurant,Supermarket,Hong Kong Restaurant,Shanghai Restaurant,Department Store
13,Scarborough,0.0,Pizza Place,Thai Restaurant,Hobby Shop,Coffee Shop,Chinese Restaurant,Bus Stop,Shopping Mall,Fried Chicken Joint,Golf Course,Pharmacy
15,Scarborough,0.0,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Thrift / Vintage Store,Other Great Outdoors,Noodle House,Coffee Shop,Sandwich Place,Grocery Store,Pharmacy
16,Scarborough,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Home Service,Construction & Landscaping,ATM,Afghan Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
63,Central Toronto,1.0,Home Service,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
